In [1]:
# 환경설정

!pip3 install transformers
!pip3 install torch
!pip3 install konlpy
!pip3 install jamo

     |████████████████████████████████| 3.1 MB 1.5 MB/s 
     |████████████████████████████████| 3.3 MB 655 kB/s 
     |████████████████████████████████| 596 kB 2.6 MB/s 
     |████████████████████████████████| 59 kB 913 kB/s 
     |████████████████████████████████| 895 kB 1.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 19.4 MB 63 kB/s 
     |████████████████████████████████| 448 kB 2.9 MB/s 
     |████████████████████████████████| 86 kB 3.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
# 시험데이터셋 수집

!wget https://raw.githubusercontent.com/aifactory-team/hanryubank/main/dataset_test.csv

--2021-11-19 08:14:08--  https://raw.githubusercontent.com/aifactory-team/hanryubank/main/dataset_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27277 (27K) [text/plain]
Saving to: ‘dataset_test.csv’

dataset_test.csv    100%[===================>]  26.64K  --.-KB/s    in 0.003s  

2021-11-19 08:14:08 (9.11 MB/s) - ‘dataset_test.csv’ saved [27277/27277]



In [7]:
# 모듈 설정

from transformers import ElectraTokenizer, ElectraForQuestionAnswering, pipeline
import pandas as pd

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
model = ElectraForQuestionAnswering.from_pretrained("monologg/koelectra-small-v2-distilled-korquad-384")
qa = pipeline("question-answering", tokenizer=tokenizer, model=model)

import konlpy
okt = konlpy.tag.Okt()

from jamo import h2j, j2hcj

vowels = ['ㅣ', 'ㅔ', 'ㅐ', 'ㅏ', 'ㅜ', 'ㅗ', 'ㅓ', 'ㅡ', 'ㅟ', 'ㅚ', 'ㅑ', 'ㅕ', 'ㅛ', 'ㅠ', 'ㅒ', 'ㅖ', 'ㅘ', 'ㅝ', 'ㅙ', 'ㅞ', 'ㅢ']

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:02<?, ?B/s]

Downloading:   0%|          | 0.00/472 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.3M [00:01<?, ?B/s]

In [25]:
# 입력된 질문(query)에 해당하는 유사한 질문을 찾고, 그 유사한 질문에 해당하는 지문을 검색하는 함수

def get_context_and_question(query):

    # 데이터베이스 로딩
    df = pd.read_csv('dataset_test.csv')

    # 데이터베이스에서 사용자 질의와 유사한 질문 획득
    result = df[df['Q1 (질문1)'] == query]

    # 유사 질문에 해당하는 지문과 질문 획득
    context = result['Context (지문)'].item()
    question = result['Q1 (질문1)'].item()

    return context, question

In [26]:
# 지문과 질문을 입력하여 정답을 얻는 함수

def get_answer_with_context(context, question):
    result = qa({"question": question, "context": context})
    score = result["score"]
    start = result["start"]
    end = result["end"]
    answer = result["answer"]
    return answer, start, end, score

In [27]:
# 해답의 어미를 바꾸는 함수

def modify_answer(before_answer):

    document = before_answer
    
    clean_words = []

    for word in okt.pos(document, stem=True):
        if word[1] not in ['Josa', 'Eomi', 'Punctuation']:
            clean_words.append(word[0])
    
    last_char = j2hcj(h2j(clean_words[-1]))[-1]

    if last_char in vowels:
        clean_words.append('라네')
    else:
        clean_words.append('이라네')

    after_answer = ''.join(clean_words)

    return after_answer

In [28]:
# 질문으로 답 얻기

def print_answer(query):

    # 질문에 해당하는 데이터베이스에 저장된 유사 질문 및 지문 획득
    context, question = get_context_and_question(query)

    # 지문과 질문을 입력해서 정답을 얻음
    answer, start, end, score = get_answer(context, question)

    # 출력
    print("query: " + query)
    print("question: " + question)
    print("predict: {0} ({1},{2},{3})".format(answer, start, end, score))
    print("service: " + modify_answer(answer))

In [29]:
if __name__ == "__main__":

    print_answer('훈민정음을 창제한 이유')
    print_answer('조세에 관하여 무엇을 시행했는가')

query: 훈민정음을 창제한 이유
question: 훈민정음을 창제한 이유
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네
query: 조세에 관하여 무엇을 시행했는가
question: 조세에 관하여 무엇을 시행했는가
predict: 여론 조사를 (30,36,0.9997721314430237)
service: 여론조사라네


In [30]:
def full_test():
    df = pd.read_csv('dataset_test.csv')
    
    for idx, row in df.iterrows():
        
        it_question = row['Q1 (질문1)']
        
        print("[" + str(idx) + "]")
        print_answer(it_question)

In [31]:
full_test()

[0]
query: 세종대왕이 무엇을 만들었는가
question: 세종대왕이 무엇을 만들었는가
predict: 훈민정음을 (20,25,0.9984866976737976)
service: 훈민정음이라네
[1]
query: 조세에 관하여 무엇을 시행했는가
question: 조세에 관하여 무엇을 시행했는가
predict: 여론 조사를 (30,36,0.9997721314430237)
service: 여론조사라네
[2]
query: 훈민정음을 창제한 이유
question: 훈민정음을 창제한 이유
predict: 애민 정신을 (320,326,0.6614944338798523)
service: 애민정신이라네
[3]
query: 농부를 위해 만든 것은
question: 농부를 위해 만든 것은
predict: 농사직설을 (124,129,0.9998931884765625)
service: 농사직설이라네
[4]
query: 어렸을 때부터 똑똑했던 이유는
question: 어렸을 때부터 똑똑했던 이유는
predict: 책읽기를 (48,52,0.9598397016525269)
service: 책읽다라네
[5]
query: 여가 생활에 어떤 것을 했을까?
question: 여가 생활에 어떤 것을 했을까?
predict: 음악을 (132,135,0.9993412494659424)
service: 음악이라네
[6]
query: 한글의 다른 이름은 무엇인가?
question: 한글의 다른 이름은 무엇인가?
predict: 언문이라 (16,20,0.9977233409881592)
service: 언문이라네
[7]
query: 언제 공식적으로 한글이 되었나
question: 언제 공식적으로 한글이 되었나
predict: 1894년 (0,5,0.9279113411903381)
service: 1894년이라네
[8]
query: 한글이라고 이름을 바꾼 사람은
question: 한글이라고 이름을 바꾼 사람은
predict: 주시경이 (310,314,0.9978508353233337)
service:

Process ForkPoolWorker-35:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/usr/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


KeyboardInterrupt: ignored